<a href="https://colab.research.google.com/github/Hero0963/kaggle/blob/main/kaggle_binary_classification_of_machine_failures/machine_failures_feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

rm: cannot remove '/root/.kaggle': No such file or directory


In [ ]:
!kaggle competitions download -c playground-series-s3e17

 86% 3.00M/3.49M [00:01<00:00, 3.23MB/s]
100% 3.49M/3.49M [00:01<00:00, 2.81MB/s]


In [ ]:
import zipfile
import pandas as pd

# 解壓縮zip檔案
with zipfile.ZipFile('/content/playground-series-s3e17.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/extracted_folder')


In [ ]:
df_train = pd.read_csv('/content/extracted_folder/train.csv')

In [ ]:
df_test = pd.read_csv('/content/extracted_folder/test.csv')

In [ ]:
df_all = pd.concat([df_train, df_test], ignore_index=True)

In [ ]:
features = df_all.drop('Machine failure', axis=1)
label = df_all['Machine failure']

In [ ]:
X = features
y = label

In [ ]:
selected_features = ['HDF', 'OSF', 'PWF', 'TWF', 'Torque [Nm]']

In [ ]:
X_s = X[selected_features]

#Normalize Features

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler


# 標準化
scaler = StandardScaler()
features_normalized_standardized  = X_s.copy()
features_normalized_standardized = scaler.fit_transform(features_normalized_standardized)

In [ ]:
df_train_data = df_all[df_all['Machine failure'].notnull()]
df_test_data = df_all[df_all['Machine failure'].isnull()]

In [ ]:
X = df_train_data[selected_features]
y = df_train_data['Machine failure']

# OverSampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler, SMOTE

# 創建Oversampling的實例
# 隨機過採樣（Random Over Sampling）
ros = RandomOverSampler(random_state=42)

# SMOTE過採樣（Synthetic Minority Over-sampling Technique）
smote = SMOTE(random_state=42)

# 使用Oversampling方法處理不平衡數據集
X_ros, y_ros = ros.fit_resample(X, y)  # 使用Random Over Sampling
X_smote, y_smote = smote.fit_resample(X, y)  # 使用SMOTE

# X_ros 和 y_ros 是經過Random Over Sampling後的新數據集
# X_smote 和 y_smote 是經過SMOTE過採樣後的新數據集

In [ ]:
# machine learning
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier

# check AUC score

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

In [ ]:
X = X_smote
y = y_smote

In [ ]:
# RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf_auc = cross_val_score(rf, X, y.values.ravel(),cv=5,scoring='roc_auc').mean()
display(rf_auc)

0.9912793043783115

#用一般資料集

In [ ]:
X = df_train_data[selected_features]
y = df_train_data['Machine failure']

In [ ]:
# RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf_auc = cross_val_score(rf, X, y.values.ravel(),cv=5,scoring='roc_auc').mean()
display(rf_auc)

0.9194061637547317

# XGBoost

In [ ]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
X = df_train_data[selected_features]
y = df_train_data['Machine failure']

In [ ]:
display(X.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136429 entries, 0 to 136428
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   HDF          136429 non-null  int64  
 1   OSF          136429 non-null  int64  
 2   PWF          136429 non-null  int64  
 3   TWF          136429 non-null  int64  
 4   Torque [Nm]  136429 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 6.2 MB


None

In [ ]:
display(X.describe())

,HDF,OSF,PWF,TWF,Torque [Nm]
count,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000
mean,0.005160,0.003958,0.002397,0.001554,40.348643
std,0.071649,0.062789,0.048899,0.039389,8.502229
min,0.000000,0.000000,0.000000,0.000000,3.800000
25%,0.000000,0.000000,0.000000,0.000000,34.600000
50%,0.000000,0.000000,0.000000,0.000000,40.400000
75%,0.000000,0.000000,0.000000,0.000000,46.100000
max,1.000000,1.000000,1.000000,1.000000,76.600000


In [ ]:
test_X = df_test_data[selected_features]

In [ ]:
rf = RandomForestClassifier(n_estimators=200)
rf.fit(X_smote, y_smote)

RandomForestClassifier(n_estimators=200)

In [ ]:
predicted_labels = rf.predict(test_X)
submission = pd.DataFrame({'id': df_test['id'], 'Machine failure': predicted_labels})
submission['Machine failure'] = submission['Machine failure'].astype(float) # convert_data_type_important
submission.to_csv('kaggle_bi_classification.csv', index=False)

#用一般資料集

In [ ]:
X = df_train_data[selected_features]
y = df_train_data['Machine failure']


rf = RandomForestClassifier(n_estimators=200)
rf.fit(X, y)

RandomForestClassifier(n_estimators=200)

In [ ]:
predicted_labels = rf.predict(test_X)
submission = pd.DataFrame({'id': df_test['id'], 'Machine failure': predicted_labels})
submission['Machine failure'] = submission['Machine failure'].astype(float) # convert_data_type_important
submission.to_csv('kaggle_bi_classification.csv', index=False)